## Experimental Workflows


In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils

microscope, settings = utils.setup_session()

### Automated Quality Control

In [ ]:

# do contrast test polishing
protocol = {
    "milling": {
        "polish": {
            "stages": [{
        "application_file": "autolamella",
        "cross_section": "CleaningCrossSection",
        "hfw": 40e-6,
        "height": 6.0e-07,
        "width": 6.0e-06,
        "depth": 4.0e-07,
        "milling_current": 6.0e-11,
        "milling_voltage": 3.0e3,
        "type": "Rectangle",
        "name": "AdaptivePolishing",
            }
        ],
        "point": {
            "x": 0.0,
            "y": 5e-6,}
        }
    },
    "options": {
        "experimental": {
            "adaptive_polishing": {
                "threshold": 100,
                "step_size": 5e-6,
                "step_limit": 10,
                "image_resolution": [3072, 2188],
                "image_line_integration": 20,
                "image_dwell_time": 100e-9,
            }

    }
}
}

from autolamella.workflows.experimental import adaptive_mill_polishing


adaptive_mill_polishing(microscope, settings, protocol)